In [1]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi

%matplotlib inline

In [2]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [3]:
dotenv.load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [5]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangm_tickers = ['AAPL', 'AMZN', 'FB', 'GOOG', 'MSFT', 'NFLX']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangm_original = alpaca.get_bars(faangm_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangm_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,118.73,118.8100,116.8600,117.34,34852374,187129,117.756760,AAPL
2015-12-02 05:00:00+00:00,117.00,118.1100,116.0800,116.28,33385643,180616,117.151198,AAPL
2015-12-03 05:00:00+00:00,116.55,116.7900,114.2200,115.20,41560785,245330,115.434888,AAPL
2015-12-04 05:00:00+00:00,115.20,119.2500,115.1100,119.03,57776977,307788,118.187290,AAPL
2015-12-07 05:00:00+00:00,118.98,119.8600,117.8100,118.28,32080754,190809,118.509111,AAPL
...,...,...,...,...,...,...,...,...
2022-04-04 04:00:00+00:00,374.00,396.0200,373.1300,391.50,7004417,145506,389.627400,NFLX
2022-04-05 04:00:00+00:00,389.46,390.0000,377.3050,380.15,4223477,110400,381.654450,NFLX
2022-04-06 04:00:00+00:00,374.15,376.4699,362.7300,368.35,4343830,134230,367.423997,NFLX


In [6]:
faangm = faangm_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol').fillna(0.0)
faangm.index = faangm.index.date
faangm.columns = faangm_tickers
faangm

,AAPL,AMZN,FB,GOOG,MSFT,NFLX
2015-12-01,117.34,679.06,107.12,767.04,55.22,125.37
2015-12-02,116.28,676.01,106.07,762.38,55.21,128.93
2015-12-03,115.20,666.25,104.38,752.54,54.20,126.81
2015-12-04,119.03,672.64,106.18,766.81,55.91,130.93
2015-12-07,118.28,669.83,105.61,763.25,55.81,125.36
...,...,...,...,...,...,...
2022-04-04,178.44,3366.93,233.89,2872.85,314.97,391.50
2022-04-05,175.06,3281.10,231.84,2821.26,310.88,380.15
2022-04-06,171.59,3175.12,223.30,2743.52,299.50,368.35
2022-04-07,172.14,3155.69,222.95,2729.30,301.37,362.15


In [7]:
kraken = krakenex.API(kraken_api_key, kraken_secret_key)

crypto_tickers = ['BTC', 'ETH', 'LUNA', 'SOL', 'XRP', 'ADA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'MATIC', 'DAI', 'LTC', 'ATOM', 'LINK', 'UNI', 'TRX', 'BCH']
kraken_tickers = ['BTCUSD', 'ETHUSD', 'LUNAUSD', 'SOLUSD', 'XRPUSD', 'ADAUSD', 'AVAXUSD', 'DOTUSD', 'DOGEUSD', 'SHIBUSD', 'MATICUSD', 'DAIUSD', 'LTCUSD', 'ATOMUSD', 'LINKUSD', 'UNIUSD', 'TRXUSD', 'BCHUSD']

#crypto_tickers = ['BTC', 'ETH', 'SOL', 'XRP', 'ADA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'MATIC', 'LTC', 'LINK', 'UNI', 'TRX', 'BCH']
#kraken_tickers = ['BTCUSD', 'ETHUSD', 'SOLUSD', 'XRPUSD', 'ADAUSD', 'AVAXUSD', 'DOTUSD', 'DOGEUSD', 'SHIBUSD', 'MATICUSD', 'LTCUSD', 'LINKUSD', 'UNIUSD', 'TRXUSD', 'BCHUSD']

pykraken = pykrakenapi.KrakenAPI(kraken)

dataframes = {}
for ticker in kraken_tickers:
    ticker_ohlc, last = pykraken.get_ohlc_data(ticker, 1440, 1500000000)
    ticker_ohlc['symbol'] = ticker
    dataframes[ticker] = ticker_ohlc.reset_index()
    time.sleep(1)

In [8]:
crypto_original = pd.DataFrame(pd.concat(dataframes.values(), join='inner')).rename(columns = {'dtime': 'timestamp'}).set_index('timestamp').sort_index()
crypto_original

,time,open,high,low,close,vwap,volume,count,symbol
timestamp,,,,,,,,,
2020-04-20,1587340800,231.800000,238.600000,216.000000,219.100000,226.490000,1.017306e+04,2910,BCHUSD
2020-04-20,1587340800,0.012922,0.013125,0.012349,0.012446,0.012677,2.581566e+06,120,TRXUSD
2020-04-20,1587340800,0.189120,0.194870,0.178550,0.181770,0.186480,1.599498e+07,3393,XRPUSD
2020-04-20,1587340800,0.035500,0.036566,0.033921,0.033990,0.034962,6.669707e+06,669,ADAUSD
2020-04-20,1587340800,180.190000,186.760000,166.130000,170.490000,175.270000,1.724604e+05,18785,ETHUSD
...,...,...,...,...,...,...,...,...,...
2022-04-09,1649462400,3193.140000,3269.120000,3183.200000,3261.620000,3219.200000,1.434077e+04,11140,ETHUSD
2022-04-09,1649462400,110.730000,113.000000,110.160000,112.690000,111.270000,1.424025e+04,2274,LTCUSD
2022-04-09,1649462400,15.122320,15.460670,15.122320,15.421910,15.322580,4.363840e+04,1314,LINKUSD


In [9]:
crypto_original.isna().sum()

time      0
open      0
high      0
low       0
close     0
vwap      0
volume    0
count     0
symbol    0
dtype: int64

In [10]:
crypto = crypto_original.drop(['time', 'open', 'high', 'low', 'vwap', 'volume', 'count'], axis = 1).pivot(columns = 'symbol')
crypto.index = crypto.index.date
crypto.columns = crypto_tickers
crypto

,BTC,ETH,LUNA,SOL,XRP,ADA,AVAX,DOT,DOGE,SHIB,MATIC,DAI,LTC,ATOM,LINK,UNI,TRX,BCH
2020-04-20,0.033990,2.2787,NaN,219.10,6839.2,1.01453,0.001960,NaN,170.49,3.44413,40.32,NaN,NaN,NaN,NaN,0.012446,NaN,0.18177
2020-04-21,0.034635,2.2768,NaN,219.90,6852.1,1.01021,0.001984,NaN,170.99,3.46270,40.65,NaN,NaN,NaN,NaN,0.012606,NaN,0.18340
2020-04-22,0.036665,2.3895,NaN,233.60,7136.9,1.01501,0.002005,NaN,183.16,3.68806,41.83,NaN,NaN,NaN,NaN,0.012969,NaN,0.18815
2020-04-23,0.040810,2.4300,NaN,237.50,7485.0,1.01643,0.002060,NaN,185.68,3.73123,42.95,NaN,NaN,NaN,NaN,0.013550,NaN,0.19283
2020-04-24,0.041764,2.7538,NaN,237.90,7507.8,1.01093,0.002084,NaN,187.72,3.78000,44.51,NaN,NaN,NaN,NaN,0.013920,NaN,0.19328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-05,1.170137,29.2892,91.02,365.47,45479.2,0.99985,0.172689,21.7782,3407.70,16.74518,122.79,116.27,1.5938,0.000027,126.82,0.069272,11.122,0.81726
2022-04-06,1.053412,26.9086,82.81,331.19,43179.4,0.99987,0.143326,19.8483,3168.44,15.40933,111.79,108.10,1.4550,0.000024,113.02,0.063056,9.959,0.75979
2022-04-07,1.089906,27.7077,87.67,336.58,43469.8,0.99991,0.146052,20.4139,3229.00,15.76737,114.11,103.59,1.5123,0.000025,118.54,0.064429,10.452,0.78705
2022-04-08,1.025486,26.8784,83.83,323.16,42282.6,0.99993,0.142546,19.4176,3193.91,15.13765,110.76,94.58,1.4335,0.000024,109.84,0.062571,9.738,0.75525


In [11]:
crypto.isna().sum()

BTC        0
ETH        0
LUNA     610
SOL        0
XRP        0
ADA        0
AVAX       0
DOT      120
DOGE       0
SHIB       0
MATIC      0
DAI      605
LTC      392
ATOM     589
LINK     423
UNI        0
TRX      178
BCH        0
dtype: int64

In [12]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangm_caps_list = [2.804, 1.616, 0.60781, 1.81, 2.245, 0.163534]
faangm_cap_portfolio = pd.DataFrame({'tickers':faangm_tickers, 'market cap weights':[market_cap / sum(faangm_caps_list) for market_cap in faangm_caps_list]})
faangm_cap_portfolio

,tickers,market cap weights
0,AAPL,0.303255
1,AMZN,0.174772
2,FB,0.065735
3,GOOG,0.195753
4,MSFT,0.242799
5,NFLX,0.017686


In [13]:
mcw_faangm_daily_returns = pd.DataFrame(faangm.pct_change().dot(list(faangm_cap_portfolio.loc[:, 'market cap weights'])))
mcw_faangm_daily_returns.columns = ['faangm']
mcw_faangm_daily_returns

,faangm
2015-12-01,NaN
2015-12-02,-0.004900
2015-12-03,-0.013646
2015-12-04,0.024839
2015-12-07,-0.005089
...,...
2022-04-04,0.024245
2022-04-05,-0.017957
2022-04-06,-0.028908
2022-04-07,0.000003


In [14]:
random_crypto_portfolios = random_portfolios_generator(crypto_tickers, 10)
random_crypto_portfolios

,ticker,random weights 1,random weights 2,random weights 3,random weights 4,random weights 5,random weights 6,random weights 7,random weights 8,random weights 9,random weights 10
0,BTC,0.055090,0.040690,0.099503,0.062523,0.071051,0.078159,0.091142,0.024125,0.051776,0.085057
1,ETH,0.102789,0.045859,0.059323,0.049394,0.010115,0.023494,0.080861,0.042989,0.010431,0.088665
2,LUNA,0.047751,0.039370,0.074565,0.089771,0.088472,0.050913,0.034252,0.048217,0.055515,0.033438
3,SOL,0.084907,0.060352,0.059137,0.071825,0.082125,0.019475,0.053783,0.023852,0.011464,0.059977
4,XRP,0.032496,0.060988,0.102869,0.060847,0.049851,0.068619,0.077648,0.063670,0.010529,0.082550
5,ADA,0.069654,0.070591,0.070404,0.005107,0.067749,0.057793,0.012268,0.094197,0.014312,0.084052
6,AVAX,0.079302,0.038151,0.038841,0.033766,0.028574,0.033756,0.062272,0.019529,0.051250,0.014337
7,DOT,0.078759,0.083054,0.080929,0.078267,0.046895,0.076732,0.017796,0.103675,0.065807,0.048467
8,DOGE,0.002970,0.053972,0.040783,0.082314,0.062622,0.071067,0.016287,0.069058,0.037877,0.069824
9,SHIB,0.019989,0.108261,0.052182,0.076880,0.073246,0.005090,0.014425,0.065409,0.108396,0.014342


In [15]:
crypto_daily_returns_list = []
for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns_list.append(crypto.pct_change().dot(list(random_crypto_portfolios.loc[:, f'random weights {n + 1}'])))

crypto_daily_returns = pd.concat(crypto_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns = crypto_daily_returns.rename(columns = {n: f'random crypto portfolio {n + 1}'})

crypto_daily_returns

,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-05,-0.013810,-0.020309,-0.023289,-0.024409,-0.024159,-0.017288,-0.012921,-0.023329,-0.018673,-0.023431
2022-04-06,-0.087144,-0.080881,-0.081907,-0.086332,-0.082880,-0.080022,-0.088960,-0.078461,-0.089497,-0.079060
2022-04-07,0.024585,0.019994,0.027305,0.024257,0.024787,0.020073,0.024326,0.021790,0.029178,0.024730
2022-04-08,-0.045160,-0.039568,-0.041357,-0.043803,-0.043184,-0.042491,-0.040544,-0.039046,-0.043734,-0.038673


In [16]:
daily_returns = pd.concat([mcw_faangm_daily_returns, crypto_daily_returns], axis = 1, join = 'inner')
daily_returns

,faangm,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,-0.012035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,-0.034858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,0.031092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,0.001825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,0.016116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.024245,-0.010814,-0.012237,-0.011011,-0.011476,-0.011341,-0.007493,-0.012628,-0.013922,-0.017316,-0.011772
2022-04-05,-0.017957,-0.013810,-0.020309,-0.023289,-0.024409,-0.024159,-0.017288,-0.012921,-0.023329,-0.018673,-0.023431
2022-04-06,-0.028908,-0.087144,-0.080881,-0.081907,-0.086332,-0.082880,-0.080022,-0.088960,-0.078461,-0.089497,-0.079060
2022-04-07,0.000003,0.024585,0.019994,0.027305,0.024257,0.024787,0.020073,0.024326,0.021790,0.029178,0.024730
